## Import dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
import numpy as np
import tensorflow.keras as K
import tensorflow.keras.backend as Kback

## Dataloader

In [ ]:
train_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255)   

train_dataset  = train_datagen.flow_from_directory(directory = 'D:/chest_xray/train',
                                                   target_size = (256,256),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   shuffle=True,
                                                   batch_size = 48)
validation_dataset  = train_datagen.flow_from_directory(directory = 'D:/chest_xray/val',
                                                   target_size = (256,256),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   shuffle=True,
                                                   batch_size = 48)


test_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255)   

test_dataset  = test_datagen.flow_from_directory(directory = 'D:/chest_xray/test',
                                                   target_size = (256,256),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   shuffle=False,
                                                   batch_size = 48)

In [ ]:
print(train_dataset.class_indices)
print(validation_dataset.class_indices)
print(test_dataset.class_indices)

## Metrics

In [ ]:
def f1_score(y_true, y_pred):
    true_positives = Kback.sum(Kback.round(Kback.clip(y_true * y_pred, 0, 1)))
    possible_positives = Kback.sum(Kback.round(Kback.clip(y_true, 0, 1)))
    predicted_positives = Kback.sum(Kback.round(Kback.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kback.epsilon())
    recall = true_positives / (possible_positives + Kback.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+Kback.epsilon())
    return f1_val

METRICS = [
      "accuracy",
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
      f1_score
]

## Model

#### Custom

In [ ]:
def SAM_avg(x):
    batch, _, _, channel = x.shape
    x = K.layers.Conv2D(channel//2, kernel_size=1, padding="same")(x)
    x = K.layers.Conv2D(channel//2, kernel_size=3, padding="same")(x)
    x = K.layers.BatchNormalization()(x)
    x = CAM(x)
    ## Average Pooling
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)
    ## Conv layer
    feats = K.layers.Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(x1)
    feats = K.layers.Multiply()([x, feats])
    return feats

def SAM_max(x):
    batch, _, _, channel = x.shape
    x = K.layers.SeparableConv2D(channel, kernel_size=1, padding="same")(x)
    x = K.layers.SeparableConv2D(channel, kernel_size=3, padding="same")(x)
    x = K.layers.BatchNormalization()(x)
    x = CAM(x)
    ## Max Pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    ## Conv layer
    feats = K.layers.Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(x2)
    feats = K.layers.Multiply()([x, feats])
    return feats

def CSSAM(x):
    x_avg = SAM_avg(x)
    x_max = SAM_max(x)
    x = K.layers.Concatenate()([x_avg, x_max])
    x = ChannelDropout(drop_ratio=0.5)(x)
    return x

def CAM(x, ratio=8):
    batch, _, _, channel = x.shape
    ## Shared layers
    l1 = K.layers.Dense(channel//ratio, activation="relu", use_bias=False)
    l2 = K.layers.Dense(channel, use_bias=False)
    ## Global Average Pooling
    x1 = K.layers.GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)
    ## Global Max Pooling
    x2 = K.layers.GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)
    ## Add both the features and pass through sigmoid
    feats = x1 + x2
    feats = K.layers.Activation("sigmoid")(feats)
    feats = K.layers.Multiply()([x, feats])
    return feats

class ChannelDropout(K.layers.Layer):
    def __init__(self, drop_ratio=0.2):
        super(ChannelDropout, self).__init__()
        self.drop_ratio = drop_ratio

    def build(self, input_shape):
        _, _, _, self.channels = input_shape
        # Initialize a trainable mask with ones
        self.mask = RichardsSigmoid(units=1)(self.add_weight("mask", shape=(1, 1, 1, self.channels), initializer="ones", trainable=True))

    def call(self, x):
        # Duplicate the mask to match the batch size
        mask = tf.tile(self.mask, [tf.shape(x)[0], 1, 1, 1])
        # Multiply the input by the mask
        x = x * mask
        num_channels_to_keep = int(self.channels // 1.25)
        sorted_x, indices = tf.nn.top_k(x, k=num_channels_to_keep, sorted=True)
        sorted_x = sorted_x[:,:,:,0:num_channels_to_keep]
        return sorted_x
    
class RichardsSigmoid(K.layers.Layer):
    def __init__(self, units=1, **kwargs):
        super(RichardsSigmoid, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        # Initialize learnable parameters: A, Q, mu
        self.A = self.add_weight(name='A', shape=(self.units,), initializer='uniform', trainable=True)
        self.Q = self.add_weight(name='Q', shape=(self.units,), initializer='uniform', trainable=True)
        self.mu = self.add_weight(name='mu', shape=(self.units,), initializer='uniform', trainable=True)

        super(RichardsSigmoid, self).build(input_shape)

    def call(self, x):
        # Richards sigmoid function
        return 1 / (1 + tf.exp(-self.A * tf.exp(-self.Q * (x - self.mu))))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.units,)

#### Deep Learner

In [ ]:
input_layer = K.Input(shape=(256,256,3))

deep_learner = K.applications.DenseNet169(include_top = False, weights = "imagenet", input_tensor = input_layer)
for layer in deep_learner.layers:
    layer.trainable = True

#### Model

In [ ]:
input_img = K.layers.Input(shape=(256,256,3)) 
feat_img = deep_learner(input_img)
feat_img = CSSAM(feat_img)
flat = K.layers.GlobalAveragePooling2D()(feat_img)
output = K.layers.Dense(2, activation='softmax')(flat)

model = K.Model(inputs=input_img, outputs=output)
optimizer = K.optimizers.Adam(lr=0.0001)
model.compile(loss=["categorical_crossentropy"], metrics=METRICS, optimizer = optimizer)
model.summary()

## Training

In [ ]:
model_checkpoint_callback = K.callbacks.ModelCheckpoint(
    filepath='saved.h5',
    monitor='val_f1_score',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
    )

history = model.fit(train_dataset,
                    epochs = 50,
                    validation_data = validation_dataset,
                    verbose = 1,
                    callbacks=[model_checkpoint_callback],
                    shuffle = True)

#### Training plots

In [ ]:
import matplotlib.pyplot as plt

def Train_Val_Plot(acc, val_acc, loss, val_loss, auc, val_auc, precision, val_precision, recall, val_recall, f1_score, val_f1_score):
    fig, axes = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle("MODEL'S METRICS VISUALIZATION")

    axes[0, 0].plot(range(1, len(acc) + 1), acc)
    axes[0, 0].plot(range(1, len(val_acc) + 1), val_acc)
    axes[0, 0].set_title('History of Accuracy')
    axes[0, 0].set_xlabel('Epochs')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend(['training', 'validation'])

    axes[0, 1].plot(range(1, len(loss) + 1), loss)
    axes[0, 1].plot(range(1, len(val_loss) + 1), val_loss)
    axes[0, 1].set_title('History of Loss')
    axes[0, 1].set_xlabel('Epochs')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend(['training', 'validation'])

    axes[0, 2].plot(range(1, len(auc) + 1), auc)
    axes[0, 2].plot(range(1, len(val_auc) + 1), val_auc)
    axes[0, 2].set_title('History of AUC')
    axes[0, 2].set_xlabel('Epochs')
    axes[0, 2].set_ylabel('AUC')
    axes[0, 2].legend(['training', 'validation'])

    axes[1, 0].plot(range(1, len(precision) + 1), precision)
    axes[1, 0].plot(range(1, len(val_precision) + 1), val_precision)
    axes[1, 0].set_title('History of Precision')
    axes[1, 0].set_xlabel('Epochs')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend(['training', 'validation'])

    axes[1, 1].plot(range(1, len(recall) + 1), recall)
    axes[1, 1].plot(range(1, len(val_recall) + 1), val_recall)
    axes[1, 1].set_title('History of Recall')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend(['training', 'validation'])

    axes[1, 2].plot(range(1, len(f1_score) + 1), f1_score)
    axes[1, 2].plot(range(1, len(val_f1_score) + 1), val_f1_score)
    axes[1, 2].set_title('History of F1 score')
    axes[1, 2].set_xlabel('Epochs')
    axes[1, 2].set_ylabel('Recall')  # Corrected from 'Recall' to 'F1 score'
    axes[1, 2].legend(['training', 'validation'])

    plt.tight_layout()
    plt.show()

# Call the function with your history data
Train_Val_Plot(history.history['accuracy'], history.history['val_accuracy'],
               history.history['loss'], history.history['val_loss'],
               history.history['auc'], history.history['val_auc'],
               history.history['precision'], history.history['val_precision'],
               history.history['recall'], history.history['val_recall'],
               history.history['f1_score'], history.history['val_f1_score'])

## Testing

#### Evaluation

In [ ]:
model.load_weights("saved.h5")
loss, accuracy, precision, recall, auc, f1_score = model.evaluate(test_dataset)
print("Accuracy", accuracy)
print("Loss", loss)
print("Precision", precision)
print("Recall", recall)
print("AUC", auc)
print("F1-score", f1_score)

#### Confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
Y_pred = model.predict_generator(test_dataset, 1157)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
disp = ConfusionMatrixDisplay(confusion_matrix(test_dataset.classes, y_pred),display_labels=['N', 'PB', 'PV'])
disp.plot()
plt.show()
print('Classification Report')
target_names = ['N', 'PB', 'PV']
print(classification_report(test_dataset.classes, y_pred, target_names=target_names))

#### Feature subspace

In [ ]:
modeller = K.Model(inputs=model.input, outputs=model.get_layer(name="global_average_pooling2d").output)
# Define the number of classes
num_classes = 2

# Initialize empty arrays for features and labels
all_features = []
all_labels = []

max_iterations = 624
i=0

# Extract features and labels from the Keras test generator
for batch_features, batch_labels in test_dataset:
    features = modeller.predict(batch_features)
    all_features.append(features)
    all_labels.append(batch_labels)
    i+=1
    if i >= max_iterations - 1:
        break

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
j_range = range(624)  # Adjust the range as needed

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Define colors for different classes
colors = ['y', 'r']

# Set the size and alpha for the points
point_size = 40
point_alpha = 0.3

for j in j_range:
    # Check if the number of samples or features is less than 3
#     if all_features[j].shape[0] < 3 or all_features[j].shape[1] < 3:
#         continue
    
    # Apply PCA to reduce the dimension to 3
    pca = PCA(n_components=3)
    
    features_pca_0 = pca.fit_transform(all_features[j])

    # Get the labels for this 'j'
    labels = all_labels[j]

    # Plot each class with circular markers and different colors
    for i in range(num_classes):
        class_indices = np.where(labels[:, i] == 1)[0]
        current_color = colors[i % len(colors)]  # Get the color for this class
        ax.scatter(features_pca_0[class_indices, 0], features_pca_0[class_indices, 1], features_pca_0[class_indices, 2], c=current_color, marker='o', s=point_size, alpha=point_alpha, ec='black')

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

plt.show()